# Movie Recommendation Algorithm

As part of this task, you are expected follow the instructions below and create a movie recommendation to make your users happy :)

### 1. We start off with the known dependencies we need

Pandas is a must. Pandas is a powerful data analysis library built for Python users. It helps us manipulate complicated data in a user-friendly manner. 

You will understand soon the convenience of it and come to love it as much as we do.

In [4]:
import pandas as pd # refer to pandas as pd
import api

### 2. Then we need to get our movies dataset

The dataset includes thousands of movies and detailed information for each one of them. Let's see how it looks like shall we? 


In [10]:
path = "./data/dataset2.csv"
df = pd.read_csv(path) # df stands for Data Frame
df.columns
#df.head(10) #uncomment to see how horrible the dataset looks
    


Index(['Unnamed: 0', 'adult', 'backdrop_path', 'budget', 'genres', 'homepage',
       'id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Unnamed: 25'],
      dtype='object')

As you can see, the dataset doesn't look very pretty, does it? That's why we need Pandas to get exactly what we need from the dataset. We need the following three columns from the dataset:
* title
* imdbID
* overview

Try running the code block below to see how to get your desired columns from a dataset.

In [68]:
df[['title', 'imdb_id', 'overview']]    
    

,title,imdb_id,overview
0,Blondie,tt0029927,Blondie and Dagwood are about to celebrate the...
1,"Peter Voss, Thief of Millions",tt0011436,
2,Love at Twenty,tt0055747,Love at Twenty unites five directors from five...
3,New World Disorder 9: Never Enough,NaN,Gee Atherton ripping the Worlds course the day...
4,Sesame Street: Elmo Loves You!,tt1656746,"Elmo is making a very, very super special surp..."
...,...,...,...
1515,Geliebte weiße Maus,tt0058134,„White mouse“ Fritz controls the traffic on Dr...
1516,Attack of the Killer Tomatoes!,tt0080391,Attack of the Killer Tomatoes is a 1978 comedy...
1517,Hindenburg Disaster: Probable Cause,tt0418729,
1518,Maus und Katz,tt0107532,


Let's create a function that returns the **title**, **imdbID** and **overview** of all the movies in the dataset.

In [15]:
def get_dataset():
    path = "./data/dataset2.csv"
    df = pd.read_csv(path)
    df = df[['title', 'imdb_id', 'overview']]
    df['overview'] = df['overview'].fillna('')
    return df

df = get_dataset()
df.head(5)

,title,imdb_id,overview
0,Blondie,tt0029927,Blondie and Dagwood are about to celebrate the...
1,"Peter Voss, Thief of Millions",tt0011436,
2,Love at Twenty,tt0055747,Love at Twenty unites five directors from five...
3,New World Disorder 9: Never Enough,NaN,Gee Atherton ripping the Worlds course the day...
4,Sesame Street: Elmo Loves You!,tt1656746,"Elmo is making a very, very super special surp..."


### 3. It's time to process our user's request

Our users have requested a feature where when they add a new movie to their favourites, they would like to get 2 new movie recommendations. You have already prepared the frontend and the backend for this task. All there is left is to:
* Get the movie plot from OMDB API using the IMDB ID of the movie
* Add the details to our dataset 
* If it already exists in the dataset, remove duplicates

#### 3.1 We will make use of the new API endpoint we created to get movie details by IMDB ID. Run the code block below and test it out!

In [25]:
response = api.get_movies_by_id("tt0029927")
response = pd.json_normalize(response) # normalize JSON data into a flat table
response

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Blondie,1938,Passed,30 Nov 1938,70 min,"Comedy, Family",Frank R. Strayer,"Richard Flournoy, Chic Young","Penny Singleton, Arthur Lake, Larry Simms",Dagwood loses his job on the eve of his and Bl...,...,N/A,6.9,793,tt0029927,movie,N/A,N/A,N/A,N/A,True


So it works! But as you can see the column names do not match with the ones that the dataset has for the same information... 

| column info | column name in dataset | column name in API response |
| --- | --- | --- |
|Title |title|Title|
|IMDB ID|imdb_id| imdbID|
|Overview | overview| Plot |

##### 3.2 Let's create a function that adds the neccessary details of the favourited movie to the dataset

Important points to consider: 
* There shouldn't be any duplicates in the dataset
* The column names should match

In [69]:
def add_plot(imdbID):
    
    # Use the get_dataset() function we created earlier 
    dataset = get_dataset()
    
    # Use the api.get_movies_by_id() to get the movie details for the given IMDB ID
    response = api.get_movies_by_id(imdbID)
    response = pd.json_normalize(response)
    
    # Get columns 'Title', 'imdbID', 'Plot' from the response 
    response = response[['Title', 'imdbID', 'Plot']]
    
    # Change column names to be 'title', 'imdb_id', 'overview'
    response.columns = ['title', 'imdb_id', 'overview']
    
    # Add the favourited movie details to the dataset
    output = pd.concat([dataset, response], ignore_index=True)
    
    # Remove the duplicate IMDB IDs from the dataset
    output.drop_duplicates(subset=['imdb_id'], inplace=True)

    return output


### 4. Create the recommendation algorithm

We will be focusing on the plot of the movies and use NLP (Natural Language Processing) to find similarities between the plot of the favourited movie and all the other movie plots in our database.

First, we need to import the necessary Python machine learning libraries that we can use to complete the task. 

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


Scikit-learn (full name for sklearn) is machine learning library for Python programming. It includes simple and efficient tools for predictive data analysis. [check here for more](https://scikit-learn.org/stable/index.html)

We will use one of it's functionalities that allows us to perform data analysis on text. We will compute pairwise similarity scores for all movies based on their plot descriptions and recommend movies based on that similarity score.

Now if you are wondering how we will achieve that, one way of doing it is to create a Term Frequency-Inverse Document Frequency (TF-IDF) matrix... 

In human words, **TF (Term Frequency)** is the relative frequency of a word in a document and is given as (term instances/total instances). 

**IDF (Inverse Document Frequency)** is the relative count of documents containing the term is given as log(number of documents/documents with term).

The overall importance of each word to the documents in which they appear is equal to TF * IDF

This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each row represents a movie, as before.This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score.

In [70]:
def get_recommendation(imdbID):
    df = add_plot(imdbID)
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['overview'])
    
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(df.index, index=df['imdb_id'])
    idx = indices[imdbID]
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:3]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    print(movie_indices)
    
    # Return the top 10 most similar movies
    return df['imdb_id'].iloc[movie_indices].values.tolist()

get_recommendation("tt0029927")


[1304, 296]


['tt0044357', 'tt0428430']